In [1]:
import csv
from datetime import timedelta
import gurobipy as gp 
from gurobipy import GRB
import networkx as nx
import random
import matplotlib.pyplot as plt
import inspect
from collections import defaultdict

from models import simple_mpc, mpc_duration_constr, lazy, column_generation, updated_mpc_duration_constr, run_column_generation
from helper import Service, hhmm2mins, mins2hhmm, fetch_data, draw_graph_with_edges, node_legal, no_overlap, create_duty_graph, extract_nodes, generate_paths, roster_statistics, solution_verify, solve_RMLP, new_duty_with_bellman_ford, count_overlaps, solve_MIP, new_duty_with_RCSP, new_duty_with_RCSP_priority, updated_new_duty_with_RCSP_priority, updated_new_duty_with_bellman_ford, new_duty_with_bellman_ford_2, branching_heuristic, new_duty_with_label_belmonford, solve_final_ip, greedy_crew_schedule_lp , solve_RMLP_test, new_duty_with_RCSP_priority_pred, dominates, new_duty_rcsp_tent_perm, reconstruct_path, pricing_rcsp_topolabel

In [2]:
import importlib
import models
import helper

importlib.reload(models)
importlib.reload(helper)

from models import simple_mpc, mpc_duration_constr, lazy, column_generation, updated_mpc_duration_constr, run_column_generation
from helper import Service, hhmm2mins, mins2hhmm, fetch_data, draw_graph_with_edges, node_legal, no_overlap, create_duty_graph, extract_nodes, generate_paths, roster_statistics, solution_verify, solve_RMLP, new_duty_with_bellman_ford, count_overlaps, solve_MIP, updated_new_duty_with_RCSP_priority,updated_new_duty_with_bellman_ford, new_duty_with_bellman_ford_2, branching_heuristic, new_duty_with_label_belmonford, solve_final_ip, greedy_crew_schedule_lp , solve_RMLP_test, new_duty_with_RCSP_priority_pred, dominates, new_duty_rcsp_tent_perm, reconstruct_path, pricing_rcsp_topolabel

In [ ]:
services, service_dict = fetch_data('./StepBackServices.csv', partial=False, rakes = 10)       # 46 maximum number of rakes
graph = create_duty_graph(services)
print(graph)

In [ ]:
draw_graph_with_edges(graph)

In [ ]:
graph1 = graph.copy()
init_duties, duty_count = mpc_duration_constr(graph1, service_dict,max_duty_duration=6*60, time_limit = 60,  show_logs = False, show_duties = False, show_roster_stats = False)
# init_duties, duty_count = updated_mpc_duration_constr(graph, service_dict,max_drive_duration=6*60,max_duty_duration=8*60, time_limit = 65,  show_logs = False, show_duties = False, show_roster_stats = False)

# init_duties, duty_count = simple_mpc(graph, service_dict, show_logs=False)
print(f"Initial number of duties: {len(init_duties)}")
cutoff_obj = len(init_duties)
print("For MPC method:", "\n")
all_covered = count_overlaps(init_duties, services)
print("\n")
# init_duties2 = init_duties.copy()
# threshold = -0.1;  # threshold for reduced cost
# # //////////////////// rmlp +lp + rcsp/////////////////////////////////////
# for _ in range(100000):
# # _ = 0
# # while True:
# #     _ += 1
#     selected_duties, duals, selected_duty_vars, obj = solve_RMLP(duties=init_duties2, services=services)
#     # tot_dur=0
#     # path, cost,tot_dur, modified_graph = updated_new_duty_with_bellman_ford(graph, duals,service_dict)
#     # path,cost, graph = new_duty_with_bellman_ford_2(graph, duals, service_dict,  max_duration=360)
#     max_duration = 6* 60 # 6 hours
#     path, cost,graph_cp = updated_new_duty_with_bellman_ford(graph, duals,service_dict,max_duration) #// promising approach
#     # path, cost, label_info = updated_new_duty_with_RCSP_priority(graph, duals, service_dict, max_resource=6*60)
#     # path, cost, label_info = new_duty_with_RCSP(graph, duals, service_dict, max_resource=max_duty_duration)
#     # path, cost, label_info = new_duty_with_label_belmonford(graph, duals, service_dict, max_duration)
#     # Check for LP convergence: if the new column's cost is above (or near) the threshold,
#     # then we consider that no significant improvement is possible.
#     if cost >= threshold:
#         print("LP converged: Reduced cost has reached the threshold (>= -0.1)")
#         break
#     print(_, obj, cost, path)
#     if path[1:-1] in init_duties2:
#         break
#     init_duties2.append(path[1:-1])

# print("num of duties after lp", len(init_duties2))
# print(f"iteration number {_} breaks......")
# print("IP starts solving.......")
# //////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
# incumbent_solution = [i for i in range(len(init_duties2))]  # or a subset based on your heuristic
# obj, selected_duties, model = solve_MIP(services, duties=init_duties2, incumbent_duties=incumbent_solution,
#                                         cutoff=cutoff_obj, mipgap=0.01, timelimit=60*6)
# # ///////////////////////////////////////////// IP starts solving ///////////////////////////////////////////////////////
# obj, selected_duties, model = solve_MIP(services, duties=init_duties2, cutoff=cutoff_obj, mipgap=0.01, timelimit=60*6)
# # obj, selected_duties, model = solve_MIP(services, duties=init_duties2,service_dict= service_dict, cutoff=cutoff_obj, mipgap=0.01, timelimit=60*6)

# if selected_duties is None:
#     print("No valid solution was found. Model status:", model.status)
# else:
#     tt = []
#     for duty_num in selected_duties:
#         tt.append(init_duties2[duty_num])
#     count_overlaps(tt, services)





In [ ]:
# Assume services, graph, service_dict, and init_duties are already defined
final_duties, iteration_logs = run_column_generation(
    services, graph, service_dict, init_duties,
    threshold=-1e-6, max_duration=6*60, max_iter=100000
)

# Now, iteration_logs is a list where each element is a tuple:
# (iteration number, LP objective value, new duty's reduced cost, generated path)
# Print out the detailed log information:
for log_entry in iteration_logs:
    iter_num, lp_obj, cost, path, message = log_entry
    print(f"Iteration {iter_num}: LP Obj = {lp_obj}, Cost = {cost}, Path = {path}, Message = {message}")

print("Total duties after column generation:", len(final_duties))


In [ ]:
# using label setting with pred and priority queue
# from helper import greedy_crew_schedule_lp , solve_RMLP_test, dominates, new_duty_rcsp_tent_perm, reconstruct_path
# ---- COLUMN GENERATION PHASE ----
# Copy the initial duties to a new list for augmentation
init_duties_test = init_duties.copy()
print("Number of duties before column generation:", len(init_duties_test))
threshold = -1e-6  # Example threshold for reduced cost convergence


for iter_num in range(100000):
    # Solve the restricted master LP (RMLP) with the current set of duties.
    # selected_duties, duals, selected_duty_vars, obj, lp_model = solve_RMLP(services, init_duties_test)
    selected_duties, duals, selected_duty_vars, obj = solve_RMLP(services, init_duties_test)
    max_duration = 6 * 60  # Maximum duty duration (360 minutes)
    
    # Generate a new duty using topo + dp
    # path, cost = pricing_rcsp_topolabel(graph, duals, service_dict, max_duration)

    # Generate a new column (duty) via label setting
    path, cost, templabels, permlabels = new_duty_rcsp_tent_perm(graph, duals, service_dict, max_duration)
    # path, cost, labelinfo = new_duty_with_RCSP_priority_pred(graph, duals, service_dict, max_duration)
    # path, cost, label_info = updated_new_duty_with_RCSP_priority(graph,duals, service_dict, max_resource=max_duration)
    
    # Print iteration details: obj (LP objective), cost (new duty's reduced cost), path.
    print(f"Iteration {iter_num}: LP Obj = {obj}, New duty cost = {cost}, Path = {path}")
    
    
    if path is None:
        print("no path found")
        break
    elif path[1:-1] in init_duties_test:
        print("Path already present, skipping.")
        continue
    elif cost >= threshold:
        print("Column generation converged: No new column with significant negative reduced cost.")
        break
    
    # Append the new duty (without the artificial source -2 and sink -1) to the pool.
    init_duties_test.append(path[1:-1])
    # test
    

In [8]:
# # //////////////////// rmlp +lp + testing reduced graph/////////////////////////////////////
# # Preprocess the graph by reducing it:
# graph2 = graph.copy()
# from helper import network_reduction
# max_duration = 6 * 60  # Maximum duty duration (360 minutes)
# reduced_graph = network_reduction(graph2,  service_dict, max_resource=max_duration)
# init_duties2 = init_duties.copy()
# print(len(init_duties))
# threshold = -0.1;  # threshold for reduced cost
# for _ in range(100000):
#     selected_duties, duals, selected_duty_vars, obj = solve_RMLP(duties=init_duties2, services=services)
#     # path, length, graph_cp = new_duty_with_bellman_ford(graph, duals)
#     max_duration = 6* 60 # 6 hours
#     # path,cost, graph = new_duty_with_bellman_ford_2(graph, duals, service_dict,  max_duration)
#     # path, cost,graph_cp = updated_new_duty_with_bellman_ford(graph, duals,service_dict,max_duration) #// promising approach
#     path, cost, templabels, permlabels = new_duty_rcsp_tent_perm(reduced_graph, duals, service_dict, max_duration)
#     # path, cost, label_info = updated_new_duty_with_RCSP_priority(graph, duals, service_dict, max_resource=max_duration)
# #   path, cost, label_info = new_duty_with_label_belmonford(graph, duals, service_dict, max_duration)
# #  path, cost, label_info = new_duty_with_RCSP_priority(graph, duals, service_dict, max_resource=max_duration)
#     # path, cost, label_info = new_duty_with_RCSP(graph, duals, service_dict, max_resource=max_duration)

#     # Check for LP convergence: if the new column's cost is above (or near) the threshold,
#     # then we consider that no significant improvement is possible.
#     if cost >= threshold:
#         print("LP converged: Reduced cost has reached the threshold (>= -0.1)")
#         break
#     print(_, obj, cost, path)
#     if path[1:-1] in init_duties2:
#         break
#     init_duties2.append(path[1:-1])

# print("num of duties after lp", len(init_duties2))
# print(f"iteration number {_} breaks......")
# # # print("IP starts solving.......")
# //////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
# incumbent_solution = [i for i in range(len(init_duties2))]  # or a subset based on your heuristic
# obj, selected_duties, model = solve_MIP(services, duties=init_duties2, incumbent_duties=incumbent_solution,
#                                         cutoff=cutoff_obj, mipgap=0.01, timelimit=60*6)
# # ///////////////////////////////////////////// IP starts solving ///////////////////////////////////////////////////////

# final_solution = branching_heuristic(init_duties2, services)
# # obj, selected_duties, model = solve_MIP(services, duties=init_duties2, cutoff=cutoff_obj, mipgap=0.01, timelimit=60*6)
# # obj, selected_duties, model = solve_MIP(services, duties=init_duties2,service_dict= service_dict, cutoff=cutoff_obj, mipgap=0.01, timelimit=60*6)

# if selected_duties is None:
#     print("No valid solution was found. Model status:", model.status)
# else:
#     tt = []
#     for duty_num in selected_duties:
#         tt.append(init_duties2[duty_num])
#     count_overlaps(tt, services)
# print("num op", len(init_duties2))


In [9]:
# init_duties3 = init_duties2.copy()
# print("num of duties after col gen", len(init_duties3))

In [ ]:
# from helper import greedy_crew_schedule_lp , solve_RMLP_test, dominates
# # ---- COLUMN GENERATION PHASE ----
# # Copy the initial duties to a new list for augmentation
# init_duties_test = init_duties.copy()
# print("Number of duties before column generation:", len(init_duties_test))
# threshold = -0.1  # Example threshold for reduced cost convergence

# for iter_num in range(100000):
#     # Solve the restricted master LP (RMLP) with the current set of duties.
#     selected_duties, duals, selected_duty_vars, obj, lp_model = solve_RMLP_test(services, init_duties_test)
#     max_duration = 6 * 60  # Maximum duty duration (360 minutes)
    
#     # Generate a new column (duty) via modified Bellman-Ford using the duals.
#     path, cost, label_info = new_duty_rcsp_tent_perm(graph, duals, service_dict, max_duration)
#     # path, cost, graph_cp = updated_new_duty_with_bellman_ford(graph, duals, service_dict, max_duration)
#     # path, cost, label_info = new_duty_with_RCSP_priority(graph, duals, service_dict, max_resource=max_duration)
    
#     # Print iteration details: obj (LP objective), cost (new duty's reduced cost), path.
#     print(f"Iteration {iter_num}: LP Obj = {obj}, New duty cost = {cost}, Path = {path}")
    
    
#     # If the new column has a reduced cost above (or near) the threshold or it's already present, stop.
#     if path is None or (path[1:-1] in init_duties_test) or cost >= threshold:
#         print("Column generation converged: No new column with significant negative reduced cost.")
#         break
    
# #     # Append the new duty (without the artificial source -2 and sink -1) to the pool.
# #     init_duties_test.append(path[1:-1])

# print("Number of duties after column generation:", len(init_duties_test))
# # print(f"Iteration number {iter_num} terminated the column generation loop.")

# # ---- FINAL IP SOLVER PHASE ----
# # Use the final set of duties to solve the master integer program.
# obj_val, selected_duties_ip, final_model = solve_final_ip(services, init_duties_test, mip_gap=0.01, time_limit=60*6)
# print("Final IP objective value (number of duties selected):", obj_val)
# print("Selected duties (indices) from final IP solution:", selected_duties_ip)

# # # ---- OPTIONAL: WARM-START HEURISTIC USING LP FRACTIONAL VALUES ----
# # Extract LP fractional values from the final LP model (from the last call to solve_RMLP).
# lp_values = {}
# for v in lp_model.getVars():
#     if v.VarName.startswith("duty_"):
#         index = int(v.VarName.split("_")[1])
#         lp_values[index] = v.X

# # Now run a greedy heuristic that uses these LP values to rank duties.
# selected_heuristic, uncovered = greedy_crew_schedule_lp(services, init_duties_test, lp_values)
# if uncovered:
#     print("The following services remain uncovered in the heuristic solution:", uncovered)
# else:
#     print("Warm-start greedy heuristic covers all services using", len(selected_heuristic), "duties.")


In [ ]:
# ////////////////////////ip

init_duties4 = init_duties_test.copy()
count = 0
for i, duty in enumerate(init_duties4):
    duty_time = 0
    for service in duty:
        for serv in services:
            if service == serv.serv_num:
                duty_time += serv.serv_dur
    
    if duty_time > 360:
        count += 1
        print(i, duty)

    # print(i, duty_time)
print("exceeding 6 hours",count)
print("num of duties after lp", len(init_duties4))
print("IP starts solving.......")
# final_solution = branching_heuristic(init_duties3, services)
obj,final_solution, model = solve_final_ip(services, init_duties4, mip_gap=0.01, time_limit=60*6)
# obj, selected_duties, model = solve_MIP(services, duties=init_duties2, cutoff=cutoff_obj, mipgap=0.01, timelimit=60*6)
# obj, selected_duties, model = solve_MIP(services, duties=init_duties2,service_dict= service_dict, cutoff=cutoff_obj, mipgap=0.01, timelimit=60*6)

if final_solution is None:
    print("No valid solution was found. ")
else:
    final_duties = []

    for duty_num in final_solution:
        final_duties.append(init_duties4[duty_num])

    print("objval ", obj)
    print("num of duties after ip", len(final_duties))
    count_overlaps(final_duties, services)

In [ ]:
l =[ 926, 528, 530, 532, 534, 536, 539, 542, 545, 548, 551, 554, 557, 560, 563, 566, 569, 572, 575, 578, 581, 584, 587, 590, 593, 596, 599, 602, 605, 608, 611, 614, 617, 620, 623, 626, 629, 632, 635, 638, 641, 644, 647, 650, 653, 656, 659, 662, 665, 668, 671, 674, 676, 678, 680, 682, 687]
totservicehr =0
for serv in l:
    totservicehr += service_dict[serv].serv_dur
print(totservicehr)

In [ ]:
print("num of duties after lp", len(init_duties2))
obj, selected_duties, model = solve_MIP(services, duties=init_duties2, cutoff=cutoff_obj, mipgap=0.001, timelimit=60*6)

tt = []
for duty_num in selected_duties:
    tt.append(init_duties2[duty_num])
count_overlaps(tt, services)

In [ ]:
# To check number of services that overlap in all selected duties
# Run after column generation

for i in range(1, 21, 2):
    tt = 0.01 * i
    init_duties, duty_count = mpc_duration_constr(graph, service_dict, time_limit = 20, show_logs = False, show_duties = False, show_roster_stats = False)
    print(f"Initial number of duties: {len(init_duties)}", "\n", "For MPC method:", "\n")
    all_covered = count_overlaps(init_duties, services)
    print("\n")
    init_duties2 = init_duties.copy()
    objectives, selected_duties, indexes = column_generation(method=1, graph = graph, services = services, init_duties= init_duties2, num_iter = 400, threshold=tt) 
    print(f"Threshold: {tt}")
    all_covered = count_overlaps(selected_duties, services)
    print("\n")
    if not all_covered:
        print("<---------------------Not all services covered in the previous iteration--------------------->")
        break

<h3>For number of rakes = 3</h3>
Threshold: 0.17<br>
Number of duties selected: 18<br>
Total number of services: 64<br>
Number of services that overlap in duties: 55<br>
Number of services covered in duties: 64<br>

<h3>For number of rakes = 4 </h3>
Threshold: 0.15<br>
Number of duties selected: 29<br>
Total number of services: 86<br>
Number of services that overlap in duties: 64<br>
Number of services covered in duties: 86<br>

<h3>For number of rakes = 5 </h3>
Threshold: 0.18<br>
Number of duties selected: 33<br>
Total number of services: 106<br>
Number of services that overlap in duties: 84<br>
Number of services covered in duties: 106<br>

In [ ]:
init_duties, duty_count = mpc_duration_constr(graph, service_dict, time_limit = 10, show_logs = False, show_duties = False, show_roster_stats = False)
print(init_duties)

In [ ]:
# block to write selected duties to csv file

output_filename = "duties_after_gen/duties_after_gen_701_all.csv"

with open(output_filename, mode="w", newline="") as file:
    # Define column headers
    fieldnames = [
        "Service Number", "Train Number", "Start Station", "Start Time (mins)", 
        "End Station", "End Time (mins)", "Direction", "Service Duration", 
        "Jurisdiction", "Stepback Train Number", "Break Duration", "Trip Duration",
        "Duty Duration"  # New column for total duty duration
    ]
    
    writer = csv.writer(file)
    writer.writerow(fieldnames)  # Write header

    for duty in selected_duties:
        # Calculate total duty duration (from first service start time to last service end time)
        valid_services = [service_dict[s] for s in duty if s in service_dict]
        if valid_services:
            duty_start_time = min(s.start_time for s in valid_services)
            duty_end_time = max(s.end_time for s in valid_services)
            total_duty_duration = duty_end_time - duty_start_time

            for service in valid_services:
                writer.writerow([
                    service.serv_num, service.train_num, service.start_stn, service.start_time,
                    service.end_stn, service.end_time, service.direction, service.serv_dur,
                    service.jurisdiction, service.stepback_train_num, service.break_dur, service.trip_dur,
                    total_duty_duration  # Add total duty duration for each service
                ])
        writer.writerow([])  # Add an empty row to separate duties

print(f"CSV file '{output_filename}' generated successfully.")

In [ ]:
# block to calculate duty times

print(indexes)
duty_times = []
for duty in selected_duties:
    duty_time = 0
    for serv in duty:
        for service in services:
            if service.serv_num == serv:
                duty_time += service.serv_dur
                break
    duty_times.append(mins2hhmm(duty_time))
# for i in range(len(selected_duties)):
#     print(selected_duties[i], duty_times[i])


In [ ]:
# block to plot objective values over iterations

iterations = range(1, len(objectives) + 1)

plt.figure(figsize=(10, 6))
plt.plot(iterations, objectives, marker='o', linestyle='--', color='b')
plt.title('Objective Values Over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Objective Value')
plt.grid(True)
plt.show()